In [1]:
import pandas as pd

csv_path = "flight_status_final_filtered.csv"
df = pd.read_csv(csv_path)

print("행 수:", len(df))
df.head()


C:\Users\Admin\AppData\Local\Temp\ipykernel_17372\499331240.py:4: DtypeWarning: Columns (6,10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(csv_path)


행 수: 1313426


,AIRLINE_ENGLISH,AIRLINE_KOREAN,AIRPORT,AIR_FLN,ARRIVED_ENG,ARRIVED_KOR,BAGGAGE_CLAIM,BOARDING_ENG,BOARDING_KOR,CITY,...,GATE,IO,LINE,LINE_CODE,RMK_ENG,RMK_KOR,STD,UFID,flight_date,is_delay
0,JEJU AIR,제주항공,CJJ,7C802,CHEONGJU,청주,1.0,JEJU,제주,CJU,...,NaN,I,국내,D,ARRIVED,도착,1040,20230430CJJI7C802,2023-04-30,0
1,JEJU AIR,제주항공,CJJ,7C850,CHEONGJU,청주,1.0,JEJU,제주,CJU,...,NaN,I,국내,D,DELAY,지연,2320,20230430CJJI7C850,2023-04-30,1
2,JEJU AIR,제주항공,CJJ,7C852,CHEONGJU,청주,1.0,JEJU,제주,CJU,...,NaN,I,국내,D,ARRIVED,도착,920,20230430CJJI7C852,2023-04-30,0
3,JEJU AIR,제주항공,CJJ,7C854,CHEONGJU,청주,1.0,JEJU,제주,CJU,...,NaN,I,국내,D,ARRIVED,도착,1840,20230430CJJI7C854,2023-04-30,0
4,JEJU AIR,제주항공,CJJ,7C856,CHEONGJU,청주,1.0,JEJU,제주,CJU,...,NaN,I,국내,D,ARRIVED,도착,1810,20230430CJJI7C856,2023-04-30,0


In [2]:
df_delay = df[df["RMK_KOR"] == "지연"].copy()

print("RMK_KOR == '지연' 데이터 수:", len(df_delay))
print(df_delay["RMK_KOR"].value_counts())


RMK_KOR == '지연' 데이터 수: 596
지연    596
Name: RMK_KOR, dtype: int64


In [3]:
def hhmm_to_minutes_safe(x):
    if pd.isna(x):
        return None

    # float → int
    try:
        x = int(float(x))
    except:
        return None

    # 이미 분 단위로 보이는 값 (0~1440)
    if x >= 1440:
        return x

    # HHMM 형식으로 보이는 경우
    hour = x // 100
    minute = x % 100

    # 비정상 값 방어
    if hour >= 24 or minute >= 60:
        return None

    return hour * 60 + minute


In [4]:
df_delay["STD_min"] = df_delay["STD"].apply(hhmm_to_minutes_safe)
df_delay["ETD_min"] = df_delay["ETD"].apply(hhmm_to_minutes_safe)


In [5]:
df_delay["delay_minutes"] = df_delay["ETD_min"] - df_delay["STD_min"]

# 자정 넘김 보정 (예: 23:50 → 00:20)
df_delay.loc[df_delay["delay_minutes"] < -720, "delay_minutes"] += 1440


In [6]:
# 혹시 모를 이상값 제거
df_delay = df_delay[df_delay["delay_minutes"] > 0]
df_delay = df_delay[df_delay["delay_minutes"] <= 300]


In [7]:
df_delay["delay_minutes"].describe(percentiles=[0.5, 0.75, 0.9, 0.95, 0.99])


count    275.000000
mean      69.476364
std       56.510919
min        5.000000
50%       65.000000
75%      100.000000
90%      133.000000
95%      166.500000
99%      291.300000
max      300.000000
Name: delay_minutes, dtype: float64

In [8]:
# =====================================================
# 📊 엑셀 확인용 데이터 생성
# =====================================================
check_cols = [
    "FLIGHT_DATE",
    "AIRLINE_KOREAN",
    "BOARDING_KOR",
    "ARRIVED_KOR",
    "STD",
    "ETD",
    "STD_min",
    "ETD_min",
    "delay_minutes",
    "RMK_KOR"
]

df_check = df_delay[check_cols].copy()


In [9]:
save_path = "flight_delay_minutes_check.xlsx"

df_check.to_excel(save_path, index=False)

print(f"✅ 지연시간 확인용 엑셀 저장 완료: {save_path}")


✅ 지연시간 확인용 엑셀 저장 완료: flight_delay_minutes_check.xlsx
